In [ ]:
%pip install --user numpy
%pip install --user matplotlib

In [3]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [599]:
class MultiLayerPerceptronClassifier:
    
    def __init__(self, hidden_layer, suppress_output = False):
        self.W = None
        self.b = None
        self.suppress_output = False
        self.hidden_layer = hidden_layer
        self.outputs = None
        
    def fit(self, X, y):
        #Se hará la suposición que todas las filas de X tienen el mismo número de columnas.
        self.build_network(len(X[0]))
        
        index = math.floor(random.random()*len(X))
        
        y_pred = self.feed_forward(X[index])
        print("FF: %s " % y_pred)
        
        self.back_propagation(0.23, learning_rate=0.01)

    
    #input_length esero entero positivo mayor a 1 que representa la cantidad de atributos de cada
    #dato de entrada (sin incluir el que se busca predecir).
    #hidden_dimensions es una lista que contiene las dimensiones de las capas escondidas de la red
    #si hidden_dimensions es [3,3,3] se crearán tres capas escondidas de 
    def build_network(self, input_length):
        
        dimensions = self.hidden_layer                   
        
        #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
        #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
        W = []  
        for i in range(len(dimensions)): 
            W.append([])
        
        
        #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
        #de la capa 0
        b = []
        for i in range(len(dimensions)): 
            b.append([])
        
        
        #todas las neuronas en la primera capa deben tener input_lenght pesos. De ahí en adelante, 
        #cada neurona en la capa j tendrá len(W[j-1]) pesos.
        layer_width = dimensions[0]
        
        for i in range(layer_width):
            W[0].append([])
            W[0][i] = [random.uniform(-1,1) for j in range(input_length)]
            b[0].append(0)
            b[0][i] = random.uniform(-1,1)
            W[0][i] = np.array([W[0][i]])
            

            
        #Inicialización aleatoria de pesos y bias entre -1 y 1
        for i in range(1, len(dimensions)):
            layer_width = dimensions[i]
            for j in range(layer_width):
                #se inicializan n valores aleatorios entre -1 y 1 donde n = len(W[i-1])
                W[i].append([])
                W[i][j] = [random.uniform(-1,1) for j in range(len(W[i-1]))]
                b[i].append(0)
                #se crea un bias aleatorio para la neurona
                b[i][j] = random.uniform(-1,1) 
            
                W[i][j] = np.array([W[i][j]])
                
        
        #Ahora se insertan los pesos y bias al output unit
        W.append([])
        W[len(W) - 1].append([])
        b.append([])
        b[len(W) - 1].append(0)
        W[len(W) - 1][0] = [random.uniform(-1,1) for j in range(len(W[len(W)-2]))]
        W[len(W) - 1][0] = np.array([W[len(W) - 1][0]])
        b[len(W) - 1][0] = random.uniform(-1,1)
            
        self.W = W
        self.b = b
        
        if not self.suppress_output:
            print("Network Built")
            print("W")
            print(W)
            print("b")
            print(b)
            print("--------------------------------------------------------------------")
            
    #X es una entrada
    #W matriz de pesos. 3D, primera dimensión capa, Segunda dimensión neurona, tercera dimensión pesos. 
    #Entonces, W[0][0] contendría el arreglo con los pesos de la neurona 0 de la capa 0.
    #l será el número de capas.
    #b es un arreglo 2D que contiene el bias de cada neurona b[0][0] contiene el bias de la neurona 0 
    #de la capa 0.
    #Algoritmo se basa en el algoritmo 6.3 de GoodFellow et al 
    #y en el algoritmo de la página 217 de Grus en Data Science from Scratch
    def feed_forward(self, x):
        
        l = len(self.hidden_layer)
        
        #Salidas de cada neurona. Primera dimensión capa, segunda neuronas
        #outputs[0][0] representa la entrada 0 de X. 
        #outputs[1][0] representa la salida 0 de la primera capa escondida.
        outputs = [np.array([x])]
        A = []
        
        for layer in range(1,l+1):
            
            layer_out = []
            A_out = []
            for neuron in range(len(self.W[layer-1])):
                #a es el resultado de las salidas de la capa anterior por los pesos de la capa actual.
                
                X = outputs[layer-1].transpose()
                W = self.W[layer-1][neuron]
                a = np.dot(W,X)[0][0] + self.b[layer-1][neuron]
                layer_out.append(self.relu(a))
                
            outputs.append(np.array([layer_out]))
            
        #Ahora se computa el output del output layer.
        X = outputs[len(self.W)-1].transpose()
        W = self.W[len(self.W)-1][0]
        np.dot(W,X)[0][0]
        self.b[len(self.W)-1][0]
        a = np.dot(W,X)[0][0] + self.b[len(self.W)-1][0]
        outputs.append(self.sigmoid(a))
        
        if not self.suppress_output:
            print("----------------------Feed_forward outputs--------------------------")
            print(outputs)
            print("--------------------------------------------------------------------")
        
        
        #Retorna el sigmoide del último output. Esto representa la probabilidad de que 
        #x pertenezca a la clase 1.
        self.outputs = outputs
        self.A = A
        return outputs[len(outputs)-1]
    
    #Relu para hidden units
    def relu(self, z):
        return max(0,z)
    
    #Sigmoide para Output unit
    def sigmoid(self, x):
        if x < 0:
            return 1 - 1/(1+math.exp(x))
        else:
            return 1/(1+math.exp(-x))
    
    def relu_derivative(self, z): 
        if z > 0:
            return 1
        else:
            return 0
    
    
    def sigmoid_derivative(self, z):
        s = self.sigmoid(z)
        return s*(1-s)
    
    #sea gradiente de la salida.
    def back_propagation(self, g, learning_rate):
        
        W = self.W
 
        #Calcular pesos de la capa de salida, se hace por aparte porque 
        #Usa función de activación sigmoide en lugar de relu.
    
        a = self.outputs[len(self.outputs)-1]
        
        f_a_prime = self.sigmoid_derivative(a)
        
        g = g*f_a_prime
        
        h = self.outputs[len(self.outputs)-2]
        
        w_grad = np.dot(g, h)
        b_grad = g
        
        W_k  = W[len(W) - 1][0]
        
        g = np.dot(W_k.transpose(), g)
        
        for i in range(len(self.outputs)-2, 0, -1):
            a = self.outputs[i][0]
            
            f_a_prime = [self.sigmoid_derivative(j) for j in a]
            f_a_prime = np.array([f_a_prime]).transpose()
            
            g = np.multiply(g, f_a_prime)
            
            h = self.outputs[i-1]
        
            w_grad = np.dot(g, h)
            b_grad = g
        
            W_k  = W[i-1]    
            W_k = self.listmatrix_to_matrix(W_k)
            g = np.dot(W_k.transpose(), g)

        self.W = W
        
        
        return None
    
    def listmatrix_to_matrix(self, matlist):
        return np.array([i[0] for i in matlist])
        

In [472]:
data_matrix = np.loadtxt(open("./msd_genre_dataset/fixed_ds.csv", "r"), delimiter=",", skiprows=0)



print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

y = data_matrix[:,len(data_matrix[0])-1]
X = np.delete(data_matrix, len(data_matrix[0])-1,1)

y = y.astype(int)


print(X)
print(y)


#Se intenta estandarizar X para lograr mejor desempeño. Sin embargo, no parece funcionar.
X = MinMaxScaler().fit_transform(X)

#Los datos del set de datos están agrupados por género. Es decir, primero están todas las filas que corresponden
#a 1 y después todas las que corresponden a -1. Se hace un shuffle para que, más tarde, en cross-validation
#no se creen unos modelos que predigan únicamente una clase.
np.random.shuffle(data_matrix)


print("X shape" + str(X.shape))
print("y shape" + str(y.shape))

Filas de la matriz: 8330
Columnas de la matriz: 31
[[-18.996       89.147        1.         ... 233.20616681 261.85070337
  240.83417734]
 [-19.347      125.825        4.         ... 227.2751235  261.64357048
  332.35653566]
 [ -9.472      121.707        4.         ... 549.49321044 481.14904868
  442.66313626]
 ...
 [ -9.494       88.976        4.         ... 519.52773394 538.89585608
  313.77593362]
 [ -7.617       67.929        3.         ... 591.85304812 598.05409088
  443.89380682]
 [-11.774       85.176        3.         ... 659.32142175 531.85019809
  607.21596134]]
[ 1  1  1 ... -1 -1 -1]
X shape(8330, 30)
y shape(8330,)


In [600]:
mlp = MultiLayerPerceptronClassifier(hidden_layer = [3,2,3], suppress_output = False)

mlp.fit(X, y)

Network Built
W
[[array([[-0.13932121,  0.26154431,  0.12722945, -0.59760968, -0.45392351,
        -0.22648387, -0.569504  , -0.50688154,  0.91308589,  0.48755504,
        -0.69997705,  0.75622059, -0.01906391, -0.80831366,  0.45441617,
        -0.54627429, -0.70085863, -0.05527424, -0.77410725, -0.65273755,
        -0.09458312,  0.14185284, -0.5499195 ,  0.92470709,  0.93921082,
         0.65276327,  0.71051165, -0.04089216, -0.92819459,  0.8771099 ]]), array([[ 0.07752813, -0.634814  , -0.79256433, -0.33462951, -0.33023806,
        -0.93181233, -0.03070471, -0.49532082,  0.6379007 ,  0.64615244,
         0.59709062,  0.92635479,  0.69017909,  0.62300727, -0.12154857,
         0.24245174,  0.44962666, -0.18395253, -0.7469736 , -0.7324934 ,
        -0.30616671,  0.76571855,  0.80568771,  0.39561133,  0.98820895,
        -0.82740776,  0.73131489, -0.40171549, -0.70206383, -0.38740353]]), array([[ 0.18434297, -0.10551313, -0.73867932,  0.42681321, -0.842221  ,
         0.19534009, -0.068